## **Optimal Transport Project - Advanced Machine Learning**

The objective of this project is to implement two different domain adaptation methods and to test them on the real-world computer vision ***Office/Caltech data set.*** The two methods are: ***subspace alignment*** and ***Entropic regularized optimal transport.***

`subspace alignment:` aims to project the labeled source and unlabeled target samples $S$ and $T$ in two subspaces spanned by their principal components so that the divergence between the two domains is minimized.

`Entropic regularized optimal transport:` It uses the *Sinkhorn-Knopp algorithm* to solve the entropic regularized OT problem between two empirical distributions of data matrices $S$ and $T$. This algorithm iteratively balances row and column sums in the transport matrix.

#### **Libraries**

In [41]:
from scipy.io import loadmat
import os
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from numpy.linalg import norm
import ot
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#### **Load Data**

For this project on two domains of the *Office/Caltech data set*: Webcam $(W)$ and DSLR $(D)$; they contain $295$ and $157$ observations respectively. The images have two data representations.

`CaffeNet4096` that contains 4096 numerical features representing the images and `Surf`has 800 features. For this project it was used the *Surf* dataset due to less computational cost and similar performance to the *CaffeNet4096*.

We then extract data and labels for each dataset.

In [42]:
surf_dir = 'Surf/'

data_webcam = loadmat(os.path.join(surf_dir, 'webcam.mat'))
X_webcam = data_webcam['fts']
y_webcam = data_webcam['labels'].flatten()

data_dslr = loadmat(os.path.join(surf_dir, 'dslr.mat'))
X_dslr = data_dslr['fts']
y_dslr = data_dslr['labels'].flatten()

print(data_webcam.keys())
print(data_dslr.keys())

num_classes_webcam = len(np.unique(y_webcam))
num_classes_dslr = len(np.unique(y_dslr))

print("Dimensions webcam dataset: \n", X_webcam.shape)
print("Number of classes: ", num_classes_webcam)
print("Dimensions dslr dataset \n", X_dslr.shape,)
print("Number of classes: ", num_classes_dslr)

dict_keys(['__header__', '__version__', '__globals__', 'fts', 'labels'])
dict_keys(['__header__', '__version__', '__globals__', 'fts', 'labels'])
Dimensions webcam dataset: 
 (295, 800)
Number of classes:  10
Dimensions dslr dataset 
 (157, 800)
Number of classes:  10


In [36]:
surf_dir = 'CaffeNet4096/'

data_webcam = loadmat(os.path.join(surf_dir, 'webcam.mat'))
X_webcam = data_webcam['fts']
y_webcam = data_webcam['labels'].flatten()

data_dslr = loadmat(os.path.join(surf_dir, 'dslr.mat'))
X_dslr = data_dslr['fts']
y_dslr = data_dslr['labels'].flatten()

print(data_webcam.keys())
print(data_dslr.keys())

num_classes_webcam = len(np.unique(y_webcam))
num_classes_dslr = len(np.unique(y_dslr))

print("Dimensions webcam dataset: \n", X_webcam.shape)
print("Number of classes: ", num_classes_webcam)
print("Dimensions dslr dataset \n", X_dslr.shape,)
print("Number of classes: ", num_classes_dslr)

dict_keys(['__header__', '__version__', '__globals__', 'labels', 'fts'])
dict_keys(['__header__', '__version__', '__globals__', 'labels', 'fts'])
Dimensions webcam dataset: 
 (295, 4096)
Number of classes:  10
Dimensions dslr dataset 
 (157, 4096)
Number of classes:  10


#### **Data normalization (features)**

In [43]:
# Initialize the scaler
scaler = StandardScaler()

# Z-score normalization 
X_webcam = scaler.fit_transform(X_webcam) 
X_dslr = scaler.fit_transform(X_dslr)

## **First Method: Subspace Alignment:**

The procedure followed in this method takes as reference the paper

The Subspace alignment method aims to project the labeled source and unlabeled target samples **S** ($n_s \times D$ matrix) and **T** ($n_t \times D$ matrix) into two subspaces spanned by their principal components, so that the divergence between the two domains is minimized.

Let the source samples **S** and target samples **T** be defined as:

$$
S \in \mathbb{R}^{n_s \times D}, \quad T \in \mathbb{R}^{n_t \times D}
$$

The methodology applied in this methos is mainly based on the approach outlined in Fernando et al.'s paper, *Unsupervised Visual Domain Adaptation Using Subspace Alignment*. The paper is available [here](https://openaccess.thecvf.com/content_iccv_2013/papers/Fernando_Unsupervised_Visual_Domain_2013_ICCV_paper.pdf).

#### **1. Finding the optimal nuber of components $d^*$ for the source $X_s \in \mathbb{R}^{D \times d}$ and target $X_t \in \mathbb{R}^{D \times d}$**

This part consists of two processes: First finding the maximum number of components $d_\text{max}$ and then applying cross-validation to find the optimal number of components $d^*$.

**Finding the maximum number of components $d_{\text{max}}$**

In this section, we need to find the principal components with the highest variance, which are denoted as $X_s \in \mathbb{R}^{D \times d}$ for the source and $X_t \in \mathbb{R}^{D \times d}$ for the target.

The $d$ paramater was tunned following the process of the paper, where it stablishes through a theorem that we can deduce a bound on the deviation between two successive eigenvalues. We can make use of this bound as a cutting rule for automatically determining the size of the subspaces.

The paper determines that given a confidence $\delta > 0$ and a fixed deviation $\gamma > 0$, we can select the maximum dimension $d_{\text{max}}$ such that:

$$
\left( \lambda_{\text{min}}^{d_{\text{max}}} - \lambda_{\text{min}}^{d_{\text{max}} + 1} \right) \geq \left( 1 + \sqrt{\frac{\ln 2 / \delta}{2}} \right) \left( \frac{16 d^{3/2} B}{\gamma \sqrt{n_{\text{min}}}} \right)
$$

For each $d \in \{ d \mid 1, \dots, d_{\text{max}} \}$, we then have the guarantee that as long as we select a subspace dimension $d$ such that $d \leq d_{\text{max}}$, the solution is stable and not over-fitting.

The values of the parameters $\gamma$, $\delta$, and $n_{\text{min}}$ were selected as follows:

- $\gamma = 10^5$: This is the same value used in the paper for adaptation from W to C. Although we are adapting from W to D, this value provided the best results.
- $\delta = 0.1$: This value represents a 90% confidence interval $(1- \gamma)$ for detecting significant shifts in eigenvalue differences. It is also consistent with the value used in the paper.
- $n_{\text{min}}$: This is the minimum sample size of the two datasets, `X_webcam` and `X_dslr`, ensuring that the stability criterion applies to both domains.

Different values were tested in this part, but the result of $d_{\text{max}}$ didn't fluctuate too much. For that reason it was decided to select the same values used in the paper.

**Cross validation to find the optimal number of dimensions $d^*$**

For this part, we chose to follow the approach outlined in the paper, which involves applying *2-fold cross-validation* using a *1-NN* classifier to evaluate the accuracy of each method on the target domain over 30 random trials. This number of trials is greater than that used in the paper because, with fewer trials, the optimal dimension $d^*$ varied significantly between runs. Using 30 random trials provided more stable results.

For each trial, we applied an unsupervised domain adaptation (DA) setting, where we randomly sampled labeled data in the source domain for training and used unlabeled data in the target domain as testing examples.

Using the optimal dimensionality $d^*$, we apply PCA to both `X_webcam` and `X_dslr`, resulting in the final reduced datasets $X_s$ and $X_t$ with shapes $(n_s, d^*)$ and $(n_t, d^*)$, respectively. This dimensionality reduction prepares the datasets for further processing in the subspace alignment approach, minimizing domain divergence with an optimal choice of dimensions.

In [44]:
def find_dmax(eigenvalues, gamma, beta, n_min, delta):  
            
    d_max = None

    for d in range(len(eigenvalues) - 1):

        # Deviation between consecutive eigenvalues
        eigenvalue_diff = eigenvalues[d] - eigenvalues[d + 1]
        
        # Eq. 5 from the paper
        rhs = (1 + np.sqrt(np.log(2 / delta) / 2)) * (((16 * (d)**(3/2)) * beta) / (gamma * np.sqrt(n_min)))
        
        # Condition to find d_max
        if eigenvalue_diff >= rhs:
            d_max = d + 1 
            
    return d_max if d_max else len(eigenvalues)  # Return d_max or max possible if not found

# PCA on both the source (webcam) and target (dslr) datasets
pca_webcam = PCA().fit(X_webcam)
pca_dslr = PCA().fit(X_dslr)

# Use the minimum number of components (in case the datasets have different dimensions)
min_components = min(len(pca_webcam.singular_values_), len(pca_dslr.singular_values_))

# Select only the first 'min_components' eigenvalues from both datasets
eigenvalues_webcam = pca_webcam.singular_values_[:min_components]
eigenvalues_dslr = pca_dslr.singular_values_[:min_components]

# Now calculate the element-wise minimum
eigenvalues = np.minimum(eigenvalues_webcam, eigenvalues_dslr)

# Set values for gamma, beta, and n_min
gamma = 1e5  # As mentioned in the paper
beta = max(np.max(np.abs(X_webcam)), np.max(np.abs(X_dslr)))
delta = 0.1  # Confidence value as per the paper
n_min = min(X_webcam.shape[0], X_dslr.shape[0])

# Calculate d_max
d_max = find_dmax(eigenvalues, gamma, beta, n_min, delta)
print(f'Maximum stable dimensionality (d_max): {d_max}')

Maximum stable dimensionality (d_max): 156


In [48]:
# Function to perform 2-fold cross-validation to select optimal d
def cross_validate_pca(Xs, y_webcam, d_max, n_trials=30):

    best_d = None
    best_accuracy = 0

    # Loop over dimensionalities from 1 to d_max
    for d in range(1, d_max + 1):

        # Apply PCA with d components on the source dataset (Xs)
        pca = PCA(n_components=d)
        Xs_reduced = pca.fit_transform(Xs)

        trial_accuracies = []

        # Repeat the process for 30 random trials
        for i in range(n_trials):

            # Randomly sample a subset of the source data for training
            X_train, X_test, y_train, y_test = train_test_split(Xs_reduced, y_webcam, test_size=0.5)            

            # Train the KNN classifier on the labeled source domain
            knn = KNeighborsClassifier(n_neighbors=1)
            knn.fit(X_train, y_train)

            # Make predictions on the source test set and calculate accuracy
            y_pred = knn.predict(X_test)
            trial_accuracy = accuracy_score(y_test, y_pred)
            
            trial_accuracies.append(trial_accuracy)

        # Calculate the average accuracy
        avg_accuracy = np.mean(trial_accuracies)

        # Update the best dimensionality if the current one has better accuracy
        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_d = d

    return best_d, best_accuracy

best_d, best_accuracy = cross_validate_pca(X_webcam, y_webcam, d_max)
print(f'Optimal dimensionality (best_d): {best_d} with cross-validated accuracy: {best_accuracy}')

Optimal dimensionality (best_d): 18 with cross-validated accuracy: 0.8488738738738739


**2. Source and target projected data in $\mathbb{R}^d$**

After determining the optimal number of dimensions $d^*$, we project the source data $S$ and target data $T$ into their respective subspaces $X_s$ and $X_t$. Following the procedure specified in the paper, this operation is performed as $\hat{S} = S X_s$ and $\hat{T} = T X_t$.


**3. Optimal alignment Matrix $M$**

Now, we we learn a linear transformation function that align the source subspace coordinate system to the target one using the *subspace alignment* aproach. The basis vectors are aligned by using a transformation matrix $M$ from $X_s$ to $X_t$ with the following formulation.

$M = X_s^T X_t$

After determining the optimal alignment matrix $M$, we compare the source data with the target data using a similarity function. The paper suggests a PSD approach that involves projecting the source data $\hat{S}$ into the aligned target space $\hat{T}$ using the transformation: $S_p = \hat{S} M$.

**4. Prediction of $\hat{S}$ labels using *1-NN* algorithm**

Finally, the projected source data in the target-aligned subspace $S_p$ is used to train a *1-NN* classifier with the original source labels `y_webcam`. The predictions are then compared with the actual labels of the target dataset, `y_dslr`, to calculate the accuracy.



In [49]:
def subspace_alignment(source_data, target_data, y_source, y_target, d):
    """
    Perform subspace alignment on the source and target datasets.

    Parameters:
    source_data (array): Source data matrix.
    target_data (array): Target data matrix.
    y_source (array): Labels for the source data.
    y_target (array): Labels for the target data.
    d (int): Dimensionality to use for PCA.

    Returns:
    float: Accuracy of the classifier on the aligned target data.
    """

    # PCA in both datasets with d
    pca_source = PCA(n_components=d)
    Xs = pca_source.fit_transform(source_data)

    pca_target = PCA(n_components=d)
    Xt = pca_target.fit_transform(target_data)

    # Projection onto their PCA components
    S_hat = source_data @ pca_source.components_.T  
    T_hat = target_data @ pca_target.components_.T  

    # Alignment matrix M 
    X_source = pca_source.components_.T  
    X_target = pca_target.components_.T  
    M = np.dot(X_source.T, X_target)  

    # Align the projected source data to the target subspace
    S_p = np.dot(S_hat, M) 

    # 1-NN classifier
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(S_p, y_source)
    y_pred = knn.predict(Xt)
    accuracy = accuracy_score(y_target, y_pred)
    
    return accuracy

#### **Prediction of target labels using *1-NN***

With the subspace alignment procedure implemented in a function that takes the source dataset $S$, target dataset $T$, and parameter $d$ as inputs and returns the accuracy of the *1-NN* classifier, we can now call this function to evaluate the accuracy on the aligned datasets.

In [50]:
accuracy = subspace_alignment(X_webcam, X_dslr, y_webcam, y_dslr, best_d)
print(f'Accuracy of the classifier after subspace alignment: {accuracy}')

Accuracy of the classifier after subspace alignment: 0.7898089171974523


#### **Prediction of target labels without projection**

Similarly, we implemented a *1-NN* algorithm using the original source data (without any alignment) and calculated the accuracy on the target datasets, which were also not aligned.

In [25]:
# PCA
pca_webcam_optimal = PCA(n_components=best_d)
Xs = pca_webcam_optimal.fit_transform(X_webcam)  # Projected source data

pca_dslr_optimal = PCA(n_components=best_d)
Xt = pca_dslr_optimal.fit_transform(X_dslr)  

# 1-NN classifier on the projected source data Xs
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(Xs, y_webcam)

# Predict the labels of the projected target data Xt
y_pred = knn.predict(Xt)

# Calculate accuracy on the target data
accuracy = accuracy_score(y_dslr, y_pred)
print(f'Accuracy on target points without subspace alignment: {accuracy}')

Accuracy on target points without subspace alignment: 0.18471337579617833


### **Entropic regularized optimal transport**

Entropic regularized optimal transport is an approach to optimal transport (OT) that adds an entropy term to the classic OT problem to make it more computationally efficient and stable. The Sinkhorn algorithm is an iterative method to solve the entropic regularized OT problem by iteratively updating the transport matrix $M$ between two data matrices $S \in \mathbb{R}^{n_s \times d}$ and $T \in \mathbb{R}^{n_t \times d}$ through matrix scaling steps.

**1. Uniform vectors *a* and *b*:** 

The vector $\mathbf{a} \in \mathbb{R}^{n_s}$ and $\mathbf{b} \in \mathbb{R}^{n_t}$ represents the uniform vectors for source and target dataset, where $n_s$ and $n_t$ are the number of observations of $S$ and $T$ respecitvely.

For this project, the chosen value for the uniform vectors are:
$\mathbf{a} = \left[\frac{1}{n_s}, \frac{1}{n_s}, \ldots, \frac{1}{n_s}\right]^T \in \mathbb{R}^{n_s}$ and $\mathbf{b} = \left[\frac{1}{n_t}, \frac{1}{n_t}, \ldots, \frac{1}{n_t}\right]^T \in \mathbb{R}^{n_t}$.

This ensures that the vectors can be interpreted as probability distributions, with each observation being equally likely.

**2. Cost Matriz calculation:**

The cost matrix $M$ is defined as the pairwise Euclidean distance between the points in the source dataset $S$ and the target dataset $T$. Mathematically, this can be expressed as:

$$M_{ij} = \| S^i - T^j \|^2$$

where $S^i$ is the $i^\text{th}$ observation from the source dataset, and $T^j$ is the $j^\text{th}$ observation from the target dataset. The resulting matrix $M \in \mathbb{R}^{n_s \times n_t}$ is then scaled for for subsequent computations.

This operation was done ussing the command `scipy.spatial.distance.cdist`

**3. Coupling matrix $\gamma$:**

The coupling matrix $\gamma$ is a matrix that represents the optimal way to transport mass from a source distribution $a$ to a target distribution $b$. For this case, $\gamma$ was calculated using the command `ot.sinkhorn(a, b, M, rege)`

The parameter $rege$ is a regularization parameter that balances the trade-off between minimizing the transportation cost and maintaining the smoothness of the coupling matrix. 

In this case, a similar aproach to the one used to find the parameter $d^*$ was employed. A range of values for the entropic regularization parameter $rege = [0.001, 0.01, 0.1, 1]$ was established. The accuracy was evaluated through a *2-fold cross-validation* using a *1-NN* classifier over 20 different random trials.  It was noted that lower values of $rege$ corresponded to higher accuracy.

Given that $S$ and $T$ datasets are not extensive, a small $rege$ value can be selected without negatively impacting the algorithm's efficiency.

**4. Transport the points from $S$ to $T$ using the coupling matrix $\gamma$:**

After tuning the hyperparameter $rege$, we proceed to transport the points from $S$ to $T$ using the coupling matrix, as shown in the equation: $S_a = \gamma T$.

**5. *1-NN* Classifier on the transported points:**

For this final step, we train a *1-NN* classifier using the transported points $S_a$ as features and the labels from `y_webcam`. We then make predictions on the target dataset $T$, and finally, we calculate the accuracy by comparing the actual target labels `y_webcam` with the predicted labels `y_dslr`.

In [30]:
def Entropic_Regularized_OT(S, T, y_train, y_val, rege):
    """
    Perform entropic regularized optimal transport to align S to T and classify with 1-NN.

    Parameters:
    - S: Source domain data matrix
    - T: Target domain data matrix
    - y_train: Labels for source data (S)
    - y_val: Labels for validation set (for accuracy calculation)
    - rege: Entropic regularization parameter for Sinkhorn algorithm

    Returns:
    - accuracy: Classification accuracy on validation set
    """
    
    # Uniform vectors for source and target
    ns = S.shape[0]
    nt = T.shape[0]
    a = np.ones(ns) / ns
    b = np.ones(nt) / nt

    # Normalized cost matrix
    M = cdist(S, T, metric='euclidean')
    M_normalized = M / M.max()

    # Coupling matrix γ
    gamma = ot.sinkhorn(a, b, M_normalized, rege)

    # Transported source points
    Sa = np.dot(gamma, T)

    # 1-NN classifier on transported source samples to predict labels on T
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(Sa, y_train)
    y_pred = knn.predict(T)
    
    # Calculate accuracy against true labels for T
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

In [31]:
# Regularization parameter range for tuning
rege_values = [0.001, 0.01, 0.1, 1]
average_accuracies = []
n_trials = 20 

# 2-Fold cross-validation with 20 trials on the source dataset
for rege in rege_values:
    trial_accuracies = []
    
    for _ in range(n_trials):
        # Randomly sample a subset of the source data for training
        X_train, X_test, y_train, y_test = train_test_split(X_webcam, y_webcam, test_size=0.5)          
        
        # Train the KNN classifier on the transported source domain
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(X_train, y_train)

        # Make predictions on the target dataset and calculate accuracy
        y_pred = knn.predict(X_test)
        trial_accuracy = accuracy_score(y_test, y_pred)
        
        trial_accuracies.append(trial_accuracy)
    
    # Average accuracy over all trials for the current `rege`
    avg_accuracy = np.mean(trial_accuracies)
    average_accuracies.append(avg_accuracy)
    print(f'Rege: {rege}, Average Accuracy: {avg_accuracy}')

# Select the best `rege` parameter
best_rege_index = np.argmax(average_accuracies)
best_rege = rege_values[best_rege_index]
best_accuracy = average_accuracies[best_rege_index]
print('-------------------------------------------')
print(f'Best rege: {best_rege}, Average accuracy over the source dataset: {best_accuracy}')

Rege: 0.001, Average Accuracy: 0.34256756756756757
Rege: 0.01, Average Accuracy: 0.3736486486486486
Rege: 0.1, Average Accuracy: 0.4
Rege: 1, Average Accuracy: 0.37432432432432433
-------------------------------------------
Best rege: 0.1, Average accuracy over the source dataset: 0.4


#### ***1-NN* Classifier on the transported points:**

In [32]:
# Final evaluation with the best `rege`
final_accuracy = Entropic_Regularized_OT(X_webcam, X_dslr, y_webcam, y_dslr, best_rege)
print(f'Final accuracy on target dataset using best rege: {final_accuracy}')

Final accuracy on target dataset using best rege: 0.802547770700637
